Merging the training and development data for the span identification subtask.

In [ ]:
import urllib
import pandas as pd
import numpy as np
from google.colab import files

TRAIN_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/train-improved-sentiwordnet-arguingfullindiv-pos.tsv?token=AFDEFD4F7VOZTUSCLUDLSUC6LEXMC'
DEV_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/dev-improved-sentiwordnet-arguingfullindiv-pos.tsv?token=AFDEFD3UUERAVPLXIMU52MS6LEXSI'
DEV_LABELS_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/train%2Bdev-improved.tsv?token=AFDEFD2UDJVAC5JW2NOOPZK6LKNCS'

In [ ]:
def get_comments(filename, url=True):
    if url:
        comments = []
        with urllib.request.urlopen(filename) as f:
            for line in f:
                if line.startswith(b'#'):
                    comments.append(line.decode("utf-8"))
                else:
                    break
        return comments
    with open(filename, 'r', encoding='utf8') as f:
        commentiter = takewhile(lambda s: s.startswith('#'), f)
        comments = list(commentiter)
    return comments

In [ ]:
comments = get_comments(TRAIN_URL)
train_df = pd.read_csv(TRAIN_URL, sep='\t', skiprows=len(comments), quoting=3)
train_input = train_df.groupby('sent_id')['token'].apply(list).to_frame()
print(train_df.head())
print(train_input.head())
print(list(train_df.columns))
print(train_df.shape[0])

In [ ]:
#delete 730081389
comments = get_comments(DEV_URL)
dev_df = pd.read_csv(DEV_URL, sep='\t', skiprows=len(comments), quoting=3)
dev_input = dev_df.groupby('sent_id')['token'].apply(list).to_frame()
print(dev_df.head())
print(dev_input.head())
print(list(dev_df.columns))
dev_df = dev_df[dev_df.document_id != 730081389]
print(dev_df.head())
print(dev_df.shape[0])

In [ ]:
dev_labels_df = pd.read_csv(DEV_LABELS_URL, sep='\t', skiprows=1, quoting=3)
print(dev_labels_df.head())

In [ ]:
result_df = dev_df.merge(dev_labels_df, how='inner', left_on=['document_id', 'token_start'], right_on=['document_id', 'token_start'])
print(result_df.head())
print(list(result_df.columns))

In [ ]:
result_df = result_df.drop(columns=['sent_id_y', 'token_end_y', 'token_y'])
result_df = result_df.rename(columns={"sent_id_x": "sent_id", "token_end_x": "token_end", "token_x": "token"})
print(list(result_df.columns))

In [ ]:
result_df_1 = result_df[['document_id', 'sent_id', 'token_start', 'token_end', 'token', 'label', 'positive', 'negative', 'arglex_any', 'ADJ', 'ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SYM', 'VERB', 'X']]
result_df_1.head()

In [ ]:
frames = [train_df, result_df_1]
train_dev_df = pd.concat(frames)
train_dev_df.shape[0]
file = train_dev_df.to_csv("train_dev_set.tsv", sep='\t')
files.download("train_dev_set.tsv") 